# Quick Start Guide - Azure AI Foundry

This notebook provides a hands-on introduction to Azure AI Foundry. You'll learn how to:
1. Initialize the AI Project client
2. List available models
3. Create a simple chat completion request
4. Create a basic AI agent
5. Handle basic error scenarios

## Prerequisites
- Completed environment setup from previous notebook
- Azure credentials configured

## Import Required Libraries and Setup

In the next cell, we'll:
1. Import the necessary Azure SDK libraries for authentication and AI Projects
2. Import standard Python libraries for environment variables and JSON handling
3. Initialize Azure credentials using DefaultAzureCredential
   - This will automatically use your logged-in Azure CLI credentials
   - Alternatively, it can use other authentication methods like environment variables or managed identity


In [4]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
import os
import json

# Initialize credentials
credential = DefaultAzureCredential()

## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.example` file to `.env` from the root directory
> 2. Update the project connection string in your `.env` file
> 3. Have a Hub and Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## Creating the AI Project Client

In the next cell, we'll create an AI Project client using the connection string from our `.env` file.
> **Note:** This example uses the synchronous client. For higher performance scenarios, you can also create an asynchronous client by importing `asyncio` and using the async methods from `AIProjectClient`.

The client will be used to:
- Connect to your Azure AI Project using the connection string
- Authenticate using Azure credentials
- Enable making inference requests to your deployed models


In [6]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

try:
    client = AIProjectClient.from_connection_string(
        conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")

✓ Successfully initialized AIProjectClient


## Create a Simple Completion
Let's try a basic completion request:

Now that we have an authenticated client, let's use it to make a chat completion request.
The code below demonstrates how to:
1. Get a ChatCompletionsClient from the azure-ai-inference package
2. Use it to make a simple completion request

We'll use the MODEL_DEPLOYMENT_NAME from our `.env` file, making it easy to switch between different
deployed models without changing code. This could be an Azure OpenAI model, Microsoft model, or other providers
that support chat completions.

> Note: Make sure you have the azure-ai-inference package installed (from requirements.txt or as mentioned in [README.md](../README.md#-quick-start))


In [8]:
from azure.ai.inference.models import UserMessage
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
# model_deployment_name = "text-embedding-3-large"
try:
    chat_client = client.inference.get_chat_completions_client()
    response = chat_client.complete(
        model=model_deployment_name, 
        messages=[UserMessage(content="How to be healthy in one sentence?")]
    )
    print(response.choices[0].message.content)
except Exception as e:
    print(f"An error occurred: {str(e)}")

To be healthy, prioritize a balanced diet, regular physical activity, sufficient sleep, and stress management while staying hydrated and maintaining strong social connections.


## Create a simple Agent

Using AI Agent Service, we can create a simple agent to answer health related questions.

Let's explore Azure AI Agent Service, a powerful tool for building intelligent agents.

Azure AI Agent Service is a fully managed service that helps developers build, deploy, and scale AI agents
without managing infrastructure. It combines large language models with tools that allow agents to:
- Answer questions using RAG (Retrieval Augmented Generation)
- Perform actions through tool calling 
- Automate complex workflows

The code below demonstrates how to:
1. Create an agent with a code interpreter tool
2. Create a conversation thread
3. Send a message requesting BMI analysis 
4. Process the request and get results
5. Save any generated visualizations

The agent will use the model specified in our .env file (MODEL_DEPLOYMENT_NAME) and will have access
to a code interpreter tool for creating visualizations. This showcases how agents can combine
natural language understanding with computational capabilities.

> The visualization will be saved as a PNG file in the same folder as this notebook.
 



In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

try:
    # Initialize the Code Interpreter Tool
    code_interpreter = CodeInterpreterTool()
    
    # Create an AI agent with the code interpreter tool
    agent = client.agents.create_agent(
        model=model_deployment_name,
        name="azure-finops-advisor",
        instructions=(
            "You are a FinOps specialist who provides cost optimization advice for Azure virtual machines. "
            "Help users understand Azure VM pricing models, recommend right-sizing strategies, "
            "identify cost-saving opportunities, and create visualizations comparing different VM options. "
            "When appropriate, provide cost estimates and ROI calculations for your recommendations."
        ),
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )    
    # Create a new conversation thread
    thread = client.agents.create_thread()
    
    # Create a message requesting BMI analysis and visualization
    message = client.agents.create_message(
        thread_id=thread.id,
        role="user",
        # content=(
        # "Compare the cost-efficiency of Standard_D4s_v3 and Standard_F8s_v2 VM sizes in West Europe for a web application with moderate CPU usage. "
        # "Create a visualization showing the monthly cost comparison with different usage patterns (8 hours/day vs 24/7). "
        # "Include recommendations for reserved instances vs pay-as-you-go pricing models and any cost optimization tips."
        # )
        content=(
            "Analyze Azure VMs in West Europe region to find a newer generation VM that would be an upgrade from a Standard_D4s_v3 VM. "
            "I need the same number of NICs, but better CPU performance and more RAM, while being cost-effective (ideally cheaper). "
            "Create a visualization comparing the current D4s_v3 with the top 3 alternatives on specs and hourly/monthly costs. "
            "Include details on cost savings opportunities with reserved instances and recommendations for the best overall value."
        )
        # content=(
        #     "Explain what FOCUS, which is FinOps open cost specification from the finops foundation, is about and provide me a summary of the key benefits. "
        # )
    )
    
    # Process the request by creating and running a thread run
    # run = client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    run = client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    # Retrieve and save any generated visualizations
    messages = client.agents.list_messages(thread_id=thread.id)
    for image_content in messages.image_contents:
        file_name = f"azure_vm_cost_analysis_{image_content.image_file.file_id}.png"
        client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Cost analysis saved as: {file_name}")
    
    # Print the analysis text from the assistant
    print(f"Messages: {messages}")
    if last_msg := messages.get_last_text_message_by_role("assistant"):
        print(f"Analysis: {last_msg.text.value}")
    
    # Cleanup by deleting the agent
    client.agents.delete_agent(agent.id)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Messages: {'object': 'list', 'data': [{'id': 'msg_r7obJP4rNaPEPDjqbw2HfNnL', 'object': 'thread.message', 'created_at': 1742510620, 'assistant_id': 'asst_d0HKhYvTpuLdzZhVU2mlxY7d', 'thread_id': 'thread_TzLww0N6L9H3VDLr9ew1ijvQ', 'run_id': 'run_xu7en05ar0IrMB4sigrDY2AF', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'FOCUS, which stands for FinOps Open Cost Specification, is an initiative by the FinOps Foundation aimed at establishing a standardized framework for cloud financial management (FinOps). The goal of FOCUS is to enable organizations to better manage their cloud spending by providing a clear and consistent method for measuring, attributing, and analyzing cloud costs. Here’s a summary of its key components and benefits:\n\n### Key Components of FOCUS:\n\n1. **Standardized Cost Attribution**: FOCUS provides guidelines on assigning cloud costs to specific teams, projects, or products, making it easier to understand who is responsible for what spending.\n\n2. 